# All Imports

In [1]:
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import os
import pandas as pd

### Unique Dates for A specific location

In [8]:

# Load CSV file
df = pd.read_csv("Karnataka_Datasets/Across/Kharif/Bellari/Data/SAR/SAR_Clean/Belllari_1_SAR_Data.csv")  # Replace with your file path

# Optional: See what columns exist
print(df.columns)

# Step 1: Choose a unique (Latitude, Longitude) pair
# You can list unique pairs like this:
unique_locations = df[['Latitude', 'Longitude']].drop_duplicates()
print("Available unique location pairs:\n", unique_locations)

# Step 2: Select one pair (example: first one)
lat, lon = unique_locations.iloc[0]['Latitude'], unique_locations.iloc[0]['Longitude']
print(f"\nSelected location: Latitude={lat}, Longitude={lon}")

# Step 3: Filter the DataFrame for this pair
filtered_df = df[(df['Latitude'] == lat) & (df['Longitude'] == lon)]

# Step 4: Print all unique date values
# Make sure your date column is named correctly (e.g., "Date" or "date")
unique_dates = filtered_df['date'].dropna().unique()
print("\nUnique dates for the selected location:")
for d in sorted(unique_dates):
    print(d)

Index(['system:index', 'Crop_Name', 'Latitude', 'Longitude', 'Pass_Direction',
       'VH', 'VV', 'date', '.geo'],
      dtype='object')
Available unique location pairs:
        Latitude  Longitude
0     15.118309  76.930739
1     15.240665  76.836838
2     15.101431  76.841903
3     15.101426  76.841905
4     15.171005  76.884255
...         ...        ...
4995  15.141400  76.868732
4996  15.141385  76.868725
4997  15.215993  76.841499
4998  15.177035  76.908172
4999  15.205486  76.893318

[5000 rows x 2 columns]

Selected location: Latitude=15.1183086, Longitude=76.9307391

Unique dates for the selected location:
2019-04-12
2019-04-24
2019-05-05
2019-05-06
2019-05-11
2019-05-18
2019-05-23
2019-05-30
2019-06-04
2019-06-10
2019-06-11
2019-06-16
2019-06-23
2019-06-28
2019-07-04
2019-07-05
2019-07-10
2019-07-16
2019-07-17
2019-07-22
2019-07-28
2019-07-29
2019-08-03
2019-08-10
2019-08-21
2019-08-22
2019-08-27
2019-09-03
2019-09-08
2019-09-15
2019-09-20
2019-09-27
2019-10-02
2019-10-09
201

### Keeping only Descending Orbit

In [2]:

# Define paths
input_dir = 'Karnataka_Datasets/Across/Kharif/Mysore/SAR/2019/'
output_dir = 'Karnataka_Datasets/Across/Kharif/Mysore/SAR/2019_Clean'
os.makedirs(output_dir, exist_ok=True)

# Process all CSV files in the directory
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(input_dir, filename)
        df = pd.read_csv(file_path)

        if 'Pass_Direction' in df.columns:
            # Strip whitespace and filter only DESCENDING
            df_filtered = df[df['Pass_Direction'].str.strip() == 'DESCENDING']

            # Save to output directory
            output_path = os.path.join(output_dir, filename)
            df_filtered.to_csv(output_path, index=False)
            print(f'Filtered {filename}: {len(df_filtered)} rows kept')
        else:
            print(f'Skipping {filename}: "Pass_Direction" column not found')

Filtered Mysore_1_SAR_Data.csv: 395000 rows kept
Filtered Mysore_2_SAR_Data.csv: 230000 rows kept
Filtered Mysore_3_SAR_Data.csv: 230000 rows kept
Filtered Mysore_4_SAR_Data.csv: 230000 rows kept
Filtered Mysore_5_SAR_Data.csv: 230000 rows kept
Filtered Mysore_6_SAR_Data.csv: 230000 rows kept
Filtered Mysore_7_SAR_Data.csv: 39744 rows kept


### Step: Flattening SAR DATA

In [22]:

# Load CSV
df = pd.read_csv("Karnataka_Datasets/Across/Kharif/Bellari/Data/SAR/SAR_Clean/Bellari_SAR_Data.csv")

# Extract VH and VV column groups
vh_cols = [col for col in df.columns if col.startswith("VH_")]
vv_cols = [col for col in df.columns if col.startswith("VV_")]

# Convert column dates
vh_dates = [datetime.strptime(col.split('_')[1], "%Y-%m-%d") for col in vh_cols]
vv_dates = [datetime.strptime(col.split('_')[1], "%Y-%m-%d") for col in vv_cols]

# Define target 12-day intervals
start_date = datetime(2019, 4, 1)
end_date = datetime(2020, 3, 31)
target_dates = []
d = start_date
while d <= end_date:
    target_dates.append(d)
    d += timedelta(days=12)

# Search window for matching: ±6 days
search_window = timedelta(days=6)

# Helper: Find nearest valid observation within ±6 days
def find_closest_or_interp(series, obs_dates, target_dates):
    # Create a time-indexed series
    s = pd.Series(series.values, index=pd.to_datetime(obs_dates))
    s = s.sort_index()
    s = s.astype(float)  # Fix interpolation warning

    output = {}
    for t in target_dates:
        # Find nearby values within ±6 days
        nearby = s[(s.index >= t - search_window) & (s.index <= t + search_window)].dropna()
        if not nearby.empty:
            # Use the observation closest in time to the target
            closest_idx = (nearby.index - t).map(abs).argmin()
            output[t.strftime('%Y-%m-%d')] = nearby.iloc[closest_idx]
        else:
            # Linear interpolate at the target date
            reindexed = s.reindex(s.index.union([t])).sort_index()
            interpolated = reindexed.interpolate(method='time')
            output[t.strftime('%Y-%m-%d')] = interpolated.get(t, None)
    return output


# Process all rows
processed = []

for idx, row in df.iterrows():
    meta = {col: row[col] for col in df.columns if not col.startswith("VH_") and not col.startswith("VV_")}

    # VH series
    vh_series = row[vh_cols]
    vh_interp = find_closest_or_interp(vh_series, vh_dates, target_dates)

    # VV series
    vv_series = row[vv_cols]
    vv_interp = find_closest_or_interp(vv_series, vv_dates, target_dates)

    row_out = meta.copy()
    for d in target_dates:
        ds = d.strftime('%Y-%m-%d')
        row_out[f'VH_{ds}'] = vh_interp[ds]
        row_out[f'VV_{ds}'] = vv_interp[ds]

    processed.append(row_out)
    print(f"Row {idx + 1}/{len(df)} processed.")

# Final DataFrame
final_df = pd.DataFrame(processed)

# Save output
final_df.to_csv("Karnataka_Datasets/Across/Kharif/Bellari/Data/SAR/SAR_Clean/Bellari_SAR_Data_Interpolated.csv", index=False)
print("Saved: sar_12day_final_interpolated.csv")

Row 1/24679 processed.
Row 2/24679 processed.
Row 3/24679 processed.
Row 4/24679 processed.
Row 5/24679 processed.
Row 6/24679 processed.
Row 7/24679 processed.
Row 8/24679 processed.
Row 9/24679 processed.
Row 10/24679 processed.
Row 11/24679 processed.
Row 12/24679 processed.
Row 13/24679 processed.
Row 14/24679 processed.
Row 15/24679 processed.
Row 16/24679 processed.
Row 17/24679 processed.
Row 18/24679 processed.
Row 19/24679 processed.
Row 20/24679 processed.
Row 21/24679 processed.
Row 22/24679 processed.
Row 23/24679 processed.
Row 24/24679 processed.
Row 25/24679 processed.
Row 26/24679 processed.
Row 27/24679 processed.
Row 28/24679 processed.
Row 29/24679 processed.
Row 30/24679 processed.
Row 31/24679 processed.
Row 32/24679 processed.
Row 33/24679 processed.
Row 34/24679 processed.
Row 35/24679 processed.
Row 36/24679 processed.
Row 37/24679 processed.
Row 38/24679 processed.
Row 39/24679 processed.
Row 40/24679 processed.
Row 41/24679 processed.
Row 42/24679 processed.
R

Row 335/24679 processed.
Row 336/24679 processed.
Row 337/24679 processed.
Row 338/24679 processed.
Row 339/24679 processed.
Row 340/24679 processed.
Row 341/24679 processed.
Row 342/24679 processed.
Row 343/24679 processed.
Row 344/24679 processed.
Row 345/24679 processed.
Row 346/24679 processed.
Row 347/24679 processed.
Row 348/24679 processed.
Row 349/24679 processed.
Row 350/24679 processed.
Row 351/24679 processed.
Row 352/24679 processed.
Row 353/24679 processed.
Row 354/24679 processed.
Row 355/24679 processed.
Row 356/24679 processed.
Row 357/24679 processed.
Row 358/24679 processed.
Row 359/24679 processed.
Row 360/24679 processed.
Row 361/24679 processed.
Row 362/24679 processed.
Row 363/24679 processed.
Row 364/24679 processed.
Row 365/24679 processed.
Row 366/24679 processed.
Row 367/24679 processed.
Row 368/24679 processed.
Row 369/24679 processed.
Row 370/24679 processed.
Row 371/24679 processed.
Row 372/24679 processed.
Row 373/24679 processed.
Row 374/24679 processed.


Row 676/24679 processed.
Row 677/24679 processed.
Row 678/24679 processed.
Row 679/24679 processed.
Row 680/24679 processed.
Row 681/24679 processed.
Row 682/24679 processed.
Row 683/24679 processed.
Row 684/24679 processed.
Row 685/24679 processed.
Row 686/24679 processed.
Row 687/24679 processed.
Row 688/24679 processed.
Row 689/24679 processed.
Row 690/24679 processed.
Row 691/24679 processed.
Row 692/24679 processed.
Row 693/24679 processed.
Row 694/24679 processed.
Row 695/24679 processed.
Row 696/24679 processed.
Row 697/24679 processed.
Row 698/24679 processed.
Row 699/24679 processed.
Row 700/24679 processed.
Row 701/24679 processed.
Row 702/24679 processed.
Row 703/24679 processed.
Row 704/24679 processed.
Row 705/24679 processed.
Row 706/24679 processed.
Row 707/24679 processed.
Row 708/24679 processed.
Row 709/24679 processed.
Row 710/24679 processed.
Row 711/24679 processed.
Row 712/24679 processed.
Row 713/24679 processed.
Row 714/24679 processed.
Row 715/24679 processed.


Row 1016/24679 processed.
Row 1017/24679 processed.
Row 1018/24679 processed.
Row 1019/24679 processed.
Row 1020/24679 processed.
Row 1021/24679 processed.
Row 1022/24679 processed.
Row 1023/24679 processed.
Row 1024/24679 processed.
Row 1025/24679 processed.
Row 1026/24679 processed.
Row 1027/24679 processed.
Row 1028/24679 processed.
Row 1029/24679 processed.
Row 1030/24679 processed.
Row 1031/24679 processed.
Row 1032/24679 processed.
Row 1033/24679 processed.
Row 1034/24679 processed.
Row 1035/24679 processed.
Row 1036/24679 processed.
Row 1037/24679 processed.
Row 1038/24679 processed.
Row 1039/24679 processed.
Row 1040/24679 processed.
Row 1041/24679 processed.
Row 1042/24679 processed.
Row 1043/24679 processed.
Row 1044/24679 processed.
Row 1045/24679 processed.
Row 1046/24679 processed.
Row 1047/24679 processed.
Row 1048/24679 processed.
Row 1049/24679 processed.
Row 1050/24679 processed.
Row 1051/24679 processed.
Row 1052/24679 processed.
Row 1053/24679 processed.
Row 1054/246

Row 1339/24679 processed.
Row 1340/24679 processed.
Row 1341/24679 processed.
Row 1342/24679 processed.
Row 1343/24679 processed.
Row 1344/24679 processed.
Row 1345/24679 processed.
Row 1346/24679 processed.
Row 1347/24679 processed.
Row 1348/24679 processed.
Row 1349/24679 processed.
Row 1350/24679 processed.
Row 1351/24679 processed.
Row 1352/24679 processed.
Row 1353/24679 processed.
Row 1354/24679 processed.
Row 1355/24679 processed.
Row 1356/24679 processed.
Row 1357/24679 processed.
Row 1358/24679 processed.
Row 1359/24679 processed.
Row 1360/24679 processed.
Row 1361/24679 processed.
Row 1362/24679 processed.
Row 1363/24679 processed.
Row 1364/24679 processed.
Row 1365/24679 processed.
Row 1366/24679 processed.
Row 1367/24679 processed.
Row 1368/24679 processed.
Row 1369/24679 processed.
Row 1370/24679 processed.
Row 1371/24679 processed.
Row 1372/24679 processed.
Row 1373/24679 processed.
Row 1374/24679 processed.
Row 1375/24679 processed.
Row 1376/24679 processed.
Row 1377/246

...

Row 23348/24679 processed.
Row 23349/24679 processed.
Row 23350/24679 processed.
Row 23351/24679 processed.
Row 23352/24679 processed.
Row 23353/24679 processed.
Row 23354/24679 processed.
Row 23355/24679 processed.
Row 23356/24679 processed.
Row 23357/24679 processed.
Row 23358/24679 processed.
Row 23359/24679 processed.
Row 23360/24679 processed.
Row 23361/24679 processed.
Row 23362/24679 processed.
Row 23363/24679 processed.
Row 23364/24679 processed.
Row 23365/24679 processed.
Row 23366/24679 processed.
Row 23367/24679 processed.
Row 23368/24679 processed.
Row 23369/24679 processed.
Row 23370/24679 processed.
Row 23371/24679 processed.
Row 23372/24679 processed.
Row 23373/24679 processed.
Row 23374/24679 processed.
Row 23375/24679 processed.
Row 23376/24679 processed.
Row 23377/24679 processed.
Row 23378/24679 processed.
Row 23379/24679 processed.
Row 23380/24679 processed.
Row 23381/24679 processed.
Row 23382/24679 processed.
Row 23383/24679 processed.
Row 23384/24679 processed.
R

Row 23652/24679 processed.
Row 23653/24679 processed.
Row 23654/24679 processed.
Row 23655/24679 processed.
Row 23656/24679 processed.
Row 23657/24679 processed.
Row 23658/24679 processed.
Row 23659/24679 processed.
Row 23660/24679 processed.
Row 23661/24679 processed.
Row 23662/24679 processed.
Row 23663/24679 processed.
Row 23664/24679 processed.
Row 23665/24679 processed.
Row 23666/24679 processed.
Row 23667/24679 processed.
Row 23668/24679 processed.
Row 23669/24679 processed.
Row 23670/24679 processed.
Row 23671/24679 processed.
Row 23672/24679 processed.
Row 23673/24679 processed.
Row 23674/24679 processed.
Row 23675/24679 processed.
Row 23676/24679 processed.
Row 23677/24679 processed.
Row 23678/24679 processed.
Row 23679/24679 processed.
Row 23680/24679 processed.
Row 23681/24679 processed.
Row 23682/24679 processed.
Row 23683/24679 processed.
Row 23684/24679 processed.
Row 23685/24679 processed.
Row 23686/24679 processed.
Row 23687/24679 processed.
Row 23688/24679 processed.
R

Row 23961/24679 processed.
Row 23962/24679 processed.
Row 23963/24679 processed.
Row 23964/24679 processed.
Row 23965/24679 processed.
Row 23966/24679 processed.
Row 23967/24679 processed.
Row 23968/24679 processed.
Row 23969/24679 processed.
Row 23970/24679 processed.
Row 23971/24679 processed.
Row 23972/24679 processed.
Row 23973/24679 processed.
Row 23974/24679 processed.
Row 23975/24679 processed.
Row 23976/24679 processed.
Row 23977/24679 processed.
Row 23978/24679 processed.
Row 23979/24679 processed.
Row 23980/24679 processed.
Row 23981/24679 processed.
Row 23982/24679 processed.
Row 23983/24679 processed.
Row 23984/24679 processed.
Row 23985/24679 processed.
Row 23986/24679 processed.
Row 23987/24679 processed.
Row 23988/24679 processed.
Row 23989/24679 processed.
Row 23990/24679 processed.
Row 23991/24679 processed.
Row 23992/24679 processed.
Row 23993/24679 processed.
Row 23994/24679 processed.
Row 23995/24679 processed.
Row 23996/24679 processed.
Row 23997/24679 processed.
R

Row 24266/24679 processed.
Row 24267/24679 processed.
Row 24268/24679 processed.
Row 24269/24679 processed.
Row 24270/24679 processed.
Row 24271/24679 processed.
Row 24272/24679 processed.
Row 24273/24679 processed.
Row 24274/24679 processed.
Row 24275/24679 processed.
Row 24276/24679 processed.
Row 24277/24679 processed.
Row 24278/24679 processed.
Row 24279/24679 processed.
Row 24280/24679 processed.
Row 24281/24679 processed.
Row 24282/24679 processed.
Row 24283/24679 processed.
Row 24284/24679 processed.
Row 24285/24679 processed.
Row 24286/24679 processed.
Row 24287/24679 processed.
Row 24288/24679 processed.
Row 24289/24679 processed.
Row 24290/24679 processed.
Row 24291/24679 processed.
Row 24292/24679 processed.
Row 24293/24679 processed.
Row 24294/24679 processed.
Row 24295/24679 processed.
Row 24296/24679 processed.
Row 24297/24679 processed.
Row 24298/24679 processed.
Row 24299/24679 processed.
Row 24300/24679 processed.
Row 24301/24679 processed.
Row 24302/24679 processed.
R

Row 24574/24679 processed.
Row 24575/24679 processed.
Row 24576/24679 processed.
Row 24577/24679 processed.
Row 24578/24679 processed.
Row 24579/24679 processed.
Row 24580/24679 processed.
Row 24581/24679 processed.
Row 24582/24679 processed.
Row 24583/24679 processed.
Row 24584/24679 processed.
Row 24585/24679 processed.
Row 24586/24679 processed.
Row 24587/24679 processed.
Row 24588/24679 processed.
Row 24589/24679 processed.
Row 24590/24679 processed.
Row 24591/24679 processed.
Row 24592/24679 processed.
Row 24593/24679 processed.
Row 24594/24679 processed.
Row 24595/24679 processed.
Row 24596/24679 processed.
Row 24597/24679 processed.
Row 24598/24679 processed.
Row 24599/24679 processed.
Row 24600/24679 processed.
Row 24601/24679 processed.
Row 24602/24679 processed.
Row 24603/24679 processed.
Row 24604/24679 processed.
Row 24605/24679 processed.
Row 24606/24679 processed.
Row 24607/24679 processed.
Row 24608/24679 processed.
Row 24609/24679 processed.
Row 24610/24679 processed.
R

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Input and Output directories
input_dir = "Karnataka_Datasets/Across/Kharif/Mandya/SAR/2021_Clean/"
output_dir = os.path.join(input_dir, "Interpolated")
os.makedirs(output_dir, exist_ok=True)

# Define 12-day intervals from April 1, 2019 to March 31, 2020
start_date = datetime(2021, 4, 1)
end_date = datetime(2022, 3, 31)
target_dates = []
d = start_date
while d <= end_date:
    target_dates.append(d)
    d += timedelta(days=12)

# ±6-day window
search_window = timedelta(days=6)

# Helper: Interpolation or closest observation
def find_closest_or_interp(series, obs_dates, target_dates):
    s = pd.Series(series.values, index=pd.to_datetime(obs_dates)).sort_index().astype(float)
    output = {}
    for t in target_dates:
        nearby = s[(s.index >= t - search_window) & (s.index <= t + search_window)].dropna()
        if not nearby.empty:
            closest_idx = (nearby.index - t).map(abs).argmin()
            output[t.strftime('%Y-%m-%d')] = nearby.iloc[closest_idx]
        else:
            reindexed = s.reindex(s.index.union([t])).sort_index()
            interpolated = reindexed.interpolate(method='time')
            output[t.strftime('%Y-%m-%d')] = interpolated.get(t, None)
    return output

# Process each CSV individually
csv_files = [f for f in os.listdir(input_dir) if f.endswith(".csv")]

for file in csv_files:
    input_path = os.path.join(input_dir, file)
    output_path = os.path.join(output_dir, f"interpolated_{file}")
    
    print(f"Processing: {file}")
    df = pd.read_csv(input_path)

    vh_cols = [col for col in df.columns if col.startswith("VH_")]
    vv_cols = [col for col in df.columns if col.startswith("VV_")]
    vh_dates = [datetime.strptime(col.split('_')[1], "%Y-%m-%d") for col in vh_cols]
    vv_dates = [datetime.strptime(col.split('_')[1], "%Y-%m-%d") for col in vv_cols]

    processed_rows = []

    for idx, row in df.iterrows():
        meta = {col: row[col] for col in df.columns if not col.startswith("VH_") and not col.startswith("VV_")}
        vh_series = row[vh_cols]
        vv_series = row[vv_cols]
        vh_interp = find_closest_or_interp(vh_series, vh_dates, target_dates)
        vv_interp = find_closest_or_interp(vv_series, vv_dates, target_dates)

        row_out = meta.copy()
        for d in target_dates:
            ds = d.strftime('%Y-%m-%d')
            row_out[f'VH_{ds}'] = vh_interp[ds]
            row_out[f'VV_{ds}'] = vv_interp[ds]

        processed_rows.append(row_out)

        if (idx + 1) % 100 == 0 or idx == len(df) - 1:
            print(f"  Row {idx + 1}/{len(df)} processed")

    final_df = pd.DataFrame(processed_rows)
    final_df.to_csv(output_path, index=False)
    print(f"Saved: {output_path}\n")


Processing: Mandya_10_SAR_Data.csv
  Row 100/86273 processed
  Row 200/86273 processed
  Row 300/86273 processed
  Row 400/86273 processed
  Row 500/86273 processed
  Row 600/86273 processed
  Row 700/86273 processed
  Row 800/86273 processed
  Row 900/86273 processed
  Row 1000/86273 processed
  Row 1100/86273 processed
  Row 1200/86273 processed
  Row 1300/86273 processed
  Row 1400/86273 processed
  Row 1500/86273 processed
  Row 1600/86273 processed
  Row 1700/86273 processed
  Row 1800/86273 processed
  Row 1900/86273 processed
  Row 2000/86273 processed
  Row 2100/86273 processed
  Row 2200/86273 processed
  Row 2300/86273 processed
  Row 2400/86273 processed
  Row 2500/86273 processed
  Row 2600/86273 processed
  Row 2700/86273 processed
  Row 2800/86273 processed
  Row 2900/86273 processed
  Row 3000/86273 processed
  Row 3100/86273 processed
  Row 3200/86273 processed
  Row 3300/86273 processed
  Row 3400/86273 processed
  Row 3500/86273 processed
  Row 3600/86273 processed
  

  Row 29700/86273 processed
  Row 29800/86273 processed
  Row 29900/86273 processed
  Row 30000/86273 processed
  Row 30100/86273 processed
  Row 30200/86273 processed
  Row 30300/86273 processed
  Row 30400/86273 processed
  Row 30500/86273 processed
  Row 30600/86273 processed
  Row 30700/86273 processed
  Row 30800/86273 processed
  Row 30900/86273 processed
  Row 31000/86273 processed
  Row 31100/86273 processed
  Row 31200/86273 processed
  Row 31300/86273 processed
  Row 31400/86273 processed
  Row 31500/86273 processed
  Row 31600/86273 processed
  Row 31700/86273 processed
  Row 31800/86273 processed
  Row 31900/86273 processed
  Row 32000/86273 processed
  Row 32100/86273 processed
  Row 32200/86273 processed
  Row 32300/86273 processed
  Row 32400/86273 processed
  Row 32500/86273 processed
  Row 32600/86273 processed
  Row 32700/86273 processed
  Row 32800/86273 processed
  Row 32900/86273 processed
  Row 33000/86273 processed
  Row 33100/86273 processed
  Row 33200/86273 pr

  Row 59000/86273 processed
  Row 59100/86273 processed
  Row 59200/86273 processed
  Row 59300/86273 processed
  Row 59400/86273 processed
  Row 59500/86273 processed
  Row 59600/86273 processed
  Row 59700/86273 processed
  Row 59800/86273 processed
  Row 59900/86273 processed
  Row 60000/86273 processed
  Row 60100/86273 processed
  Row 60200/86273 processed
  Row 60300/86273 processed
  Row 60400/86273 processed
  Row 60500/86273 processed
  Row 60600/86273 processed
  Row 60700/86273 processed
  Row 60800/86273 processed
  Row 60900/86273 processed
  Row 61000/86273 processed
  Row 61100/86273 processed
  Row 61200/86273 processed
  Row 61300/86273 processed
  Row 61400/86273 processed
  Row 61500/86273 processed
  Row 61600/86273 processed
  Row 61700/86273 processed
  Row 61800/86273 processed
  Row 61900/86273 processed
  Row 62000/86273 processed
  Row 62100/86273 processed
  Row 62200/86273 processed
  Row 62300/86273 processed
  Row 62400/86273 processed
  Row 62500/86273 pr

### Remove Duplicate VV and VH Cols

In [8]:

# Read the CSV file into a DataFrame
input_file = "Karnataka_Datasets/Across/SAR/SAR_Data_Clean.csv"
df = pd.read_csv(input_file)

# Identify all VV and VH columns (from VV_1 to VV_31 and VH_1 to VH_31)
vv_columns = [f'VV_{i}' for i in range(1, 32)]
vh_columns = [f'VH_{i}' for i in range(1, 32)]

# Combine all VV and VH columns into a list
all_columns = vv_columns + vh_columns

# Get the unique count of each column, then filter out those with only one unique value
columns_to_keep = [col for col in all_columns if df[col].nunique() > 1]

# Keep only the original columns (including Crop_Name, Latitude, and Longitude)
df_cleaned = df[['Latitude', 'Longitude', 'Crop_Name'] + columns_to_keep]

# Save the cleaned DataFrame back to a CSV file

# Save the cleaned DataFrame back to a CSV file
output_file = "Karnataka_Datasets/Across/SAR/SAR_Data_Clean.csv"
df_cleaned.to_csv(output_file, index=False)
print(f"Cleaned data saved to: {output_file}")

Cleaned data saved to: Karnataka_Datasets/Across/SAR/SAR_Data_Clean.csv
